# QKD Post-Processing: API Architecture & Cascade 
This notebook reflects the architecture of `bb84_server_client.py`. It demonstrates how the **Classical Post-Processing** phase of QKD is handled using a Request-Response (API) model.

### The Workflow:
1. **Quantum Transmission**: Simulated raw exchange of photons.
2. **Sifting (API)**: Alice and Bob compare bases via a public channel.
3. **Parameter Estimation (API)**: Bob samples a subset of the key to estimate QBER.
4. **Error Correction (Cascade API)**: Bob corrects his key using parity information from Alice.

---
## 1. Environment Setup & Raw Transmission
We start by generating random bits and bases for Alice, and simulated detections for Bob (including optical errors).

In [1]:
import numpy as np
import asyncio
import math
import random
from typing import List, Dict, Tuple

# Set seeds for consistency
np.random.seed(42)
random.seed(42)

def simulate_raw_transmission(num_qubits, error_rate=0.02):
    """Simulates the raw quantum phase of BB84."""
    # Alice's random bits and bases
    alice_bits = np.random.randint(0, 2, num_qubits).tolist()
    alice_bases = np.random.choice(['+', 'x'], num_qubits).tolist()
    
    # Bob chooses random bases
    bob_bases = np.random.choice(['+', 'x'], num_qubits).tolist()
    bob_bits = []
    
    for i in range(num_qubits):
        if alice_bases[i] == bob_bases[i]:
            # Bases match: Bob gets the bit (with potential optical error)
            bit = alice_bits[i]
            if np.random.random() < error_rate:
                bit = 1 - bit
            bob_bits.append(bit)
        else:
            # Bases mismatch: Bob gets a random bit
            bob_bits.append(np.random.randint(0, 2))
            
    return {
        "alice": {"bits": alice_bits, "bases": alice_bases},
        "bob": {"bits": bob_bits, "bases": bob_bases}
    }

NUM_QUBITS = 10000
raw_data = simulate_raw_transmission(NUM_QUBITS, error_rate=0.07)

print(f"Simulated {NUM_QUBITS} quantum transmissions.")
print(f"Alice's first 10 bits:  {raw_data['alice']['bits'][:10]}")
print(f"Alice's first 10 bases: {raw_data['alice']['bases'][:10]}")

Simulated 10000 quantum transmissions.
Alice's first 10 bits:  [0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
Alice's first 10 bases: ['+', 'x', '+', 'x', 'x', '+', 'x', 'x', '+', '+']


## 2. Alice as a Server (The Parity Oracle)
In the Server-Client model, Alice acts as an **Oracle**. She doesn't reveal her bits directly; instead, she answers questions about their parity.

Common API endpoints in a reconciliation server:
- `POST /parity`: Compute parity for a single list of indices.
- `POST /block-parity`: Compute parities for multiple blocks in one round-trip (Bulk operation).

This mimics the `AliceServer` class in `bb84_server_client.py`.

In [2]:
class AliceServer:
    def __init__(self, raw_bits, raw_bases):
        self.raw_bits = raw_bits
        self.raw_bases = raw_bases
        self.sifted_key = [] # Set during /sift
        self.requests_received = 0

    async def handle_request(self, endpoint: str, data: Dict):
        self.requests_received += 1
        
        if endpoint == "/sift":
            # Bob sends: {"bases": [(id, "+"), (id, "x"), ...]}
            bob_bases = data["bases"]
            matching_indices = []
            self.sifted_key = []
            
            for pid, b_basis in bob_bases:
                if self.raw_bases[pid] == b_basis:
                    matching_indices.append(pid)
                    self.sifted_key.append(self.raw_bits[pid])
            
            return {"matching_indices": matching_indices}

        if endpoint == "/sample":
            # Bob reveals indices he sample for QBER
            indices = data["indices"]
            revealed_bits = [self.sifted_key[i] for i in indices]
            
            # Discard sampled bits from Alice's internal sifted key
            indices_set = set(indices)
            self.sifted_key = [b for i, b in enumerate(self.sifted_key) if i not in indices_set]
            
            return {"bits": revealed_bits}

        if endpoint == "/parity":
            indices = data["indices"]
            p = 0
            for idx in indices: p ^= self.sifted_key[idx]
            return {"parity": p}

        if endpoint == "/block-parity":
            blocks = data["blocks"]
            parities = []
            for b_indices in blocks:
                p = 0
                for idx in b_indices: p ^= self.sifted_key[idx]
                parities.append(p)
            return {"parities": parities}

alice_server = AliceServer(raw_data['alice']['bits'], raw_data['alice']['bases'])
print("Alice Server online with /sift, /sample, and /parity endpoints.")

Alice Server online with /sift, /sample, and /parity endpoints.


## 3. Bob as a Client (The Protocol Engine)
Bob runs the Cascade algorithm. Since he doesn't have Alice's key, he uses an **API Client** to communicate with her.

### The Algorithm Flow:
1. Divide key into blocks.
2. Ask Alice for the parity of these blocks via `/block-parity`.
3. If parities differ, perform a binary search using `/parity` calls to find the exact bit to flip.
4. If a bit is flipped, "Cascade" back to previous passes to check if other parities were affected.

This decoupling allows the exact same Cascade logic to work whether Alice is in the same memory space or in a different country!

In [3]:
class BobClient:
    def __init__(self, raw_bits, raw_bases, server):
        self.raw_bits = raw_bits
        self.raw_bases = raw_bases
        self.server = server
        
        self.sifted_key = []
        self.working_key = [] # Key after discarding sampled bits
        self.est_qber = 0.0
        
        self.bits_revealed = 0
        self.total_corrected = 0

    async def run_sifting(self):
        # Prepare bases payload
        bases_payload = [(i, b) for i, b in enumerate(self.raw_bases)]
        response = await self.server.handle_request("/sift", {"bases": bases_payload})
        
        matches = set(response["matching_indices"])
        self.sifted_key = [self.raw_bits[i] for i in range(len(self.raw_bits)) if i in matches]
        print(f"[Bob] Sifting finished. Sifted Key Length: {len(self.sifted_key)}")

    async def run_parameter_estimation(self, sample_rate=0.2):
        n = len(self.sifted_key)
        k = int(n * sample_rate)
        sample_indices = random.sample(range(n), k)
        
        response = await self.server.handle_request("/sample", {"indices": sample_indices})
        alice_sample = response["bits"]
        
        errors = sum(1 for i, idx in enumerate(sample_indices) if self.sifted_key[idx] != alice_sample[i])
        self.est_qber = errors / k if k > 0 else 0.05
        
        # Discard sampled bits
        indices_set = set(sample_indices)
        self.working_key = [b for i, b in enumerate(self.sifted_key) if i not in indices_set]
        
        print(f"[Bob] Estimated QBER: {self.est_qber:.2%}. Working Key Length: {len(self.working_key)}")

    # -- Cascade Methods --
    
    async def get_alice_parity(self, indices):
        resp = await self.server.handle_request("/parity", {"indices": indices})
        self.bits_revealed += 1
        return resp["parity"]

    async def get_alice_block_parities(self, blocks):
        resp = await self.server.handle_request("/block-parity", {"blocks": blocks})
        self.bits_revealed += len(blocks)
        return resp["parities"]

    def local_parity(self, indices):
        p = 0
        for idx in indices: p ^= self.working_key[idx]
        return p

    async def binary_search(self, block_indices):
        beg, end = 0, len(block_indices) - 1
        while beg < end:
            mid = (beg + end) // 2
            left = block_indices[beg : mid + 1]
            alice_p = await self.get_alice_parity(left)
            if alice_p != self.local_parity(left):
                end = mid
            else:
                beg = mid + 1
        
        idx = block_indices[beg]
        self.working_key[idx] = 1 - self.working_key[idx]
        self.total_corrected += 1

    async def run_cascade(self, num_passes=4):
        print(f"[Bob] Starting Cascade with {num_passes} passes...")
        for p in range(1, num_passes + 1):
            # Block size logic
            if p == 1: block_size = int(0.73 / self.est_qber) if self.est_qber > 0 else 8
            else: block_size = block_size * 2
            
            n = len(self.working_key)
            indices = list(range(n))
            random.shuffle(indices)
            blocks = [indices[i : i+block_size] for i in range(0, n, block_size)]
            
            alice_ps = await self.get_alice_block_parities(blocks)
            for i, block in enumerate(blocks):
                if alice_ps[i] != self.local_parity(block):
                    await self.binary_search(block)
            
            errs = sum(1 for a, b in zip(alice_server.sifted_key, self.working_key) if a != b)
            print(f"  Pass {p} complete. Remaining Errors: {errs}")

bob_client = BobClient(raw_data['bob']['bits'], raw_data['bob']['bases'], alice_server)
print("Bob Client initialized.")

Bob Client initialized.


## 4. Execution of the Protocol
We will run 4 passes of the protocol with doubling block sizes. 

- Pass 1: Block size 16 (targeting 5% QBER)
- Pass 2: Block size 32
- Pass 3: Block size 64
- Pass 4: Block size 128

Note how Bob's key gradually converges to Alice's.

In [4]:
async def start_full_protocol():
    print("=== STARTING QKD POST-PROCESSING ===")
    
    # 1. Sifting
    await bob_client.run_sifting()
    
    # 2. Parameter Estimation
    await bob_client.run_parameter_estimation(sample_rate=0.2)
    
    # Store state before cascade for comparison
    bob_key_before_ec = list(bob_client.working_key)
    
    # 3. Error Correction (Cascade)
    await bob_client.run_cascade(num_passes=4)
    
    print("\n=== FINAL REPORT ===")
    print(f"Alice's Final Key Length: {len(alice_server.sifted_key)}")
    print(f"Bob's Final Key Length:   {len(bob_client.working_key)}")
    print(f"Errors Corrected:         {bob_client.total_corrected}")
    print(f"Parity Bits Revealed:     {bob_client.bits_revealed}")
    print(f"Success:                  {alice_server.sifted_key == bob_client.working_key}")
    
    return bob_key_before_ec

# Note: We need to define this for the next cell
bob_pre_ec = await start_full_protocol()

=== STARTING QKD POST-PROCESSING ===
[Bob] Sifting finished. Sifted Key Length: 4948
[Bob] Estimated QBER: 6.27%. Working Key Length: 3959
[Bob] Starting Cascade with 4 passes...
  Pass 1 complete. Remaining Errors: 124
  Pass 2 complete. Remaining Errors: 64
  Pass 3 complete. Remaining Errors: 28
  Pass 4 complete. Remaining Errors: 12

=== FINAL REPORT ===
Alice's Final Key Length: 3959
Bob's Final Key Length:   3959
Errors Corrected:         258
Parity Bits Revealed:     1755
Success:                  False


## 5. Visualizing the "Information Exchange"
The API architecture highlights how much information is exchanged. Every `log2(N)` calls during a binary search pinpoint exactly 1 error but also reveal 1 bit of parity to Eve.

Here is the "Status" table for the first 25 bits:
- **OK**: Matched from the start.
- **FIXED**: Corrected by Bob's protocol.
- **ERROR**: Failed to correct (though Cascade is 100% efficient if run enough times).

In [5]:
def print_final_comparison(alice, bob_pre, bob_post, limit=25):
    print(f"{'Idx':<4} | {'Alice':<5} | {'Bob Pre-EC':<10} | {'Bob Final':<10} | {'Status'}")
    print("-" * 55)
    for i in range(min(limit, len(alice))):
        a = alice[i]
        b_pre = bob_pre[i]
        b_post = bob_post[i]
        
        status = "OK"
        if a != b_pre and a == b_post: status = "FIXED"
        elif a != b_pre and a != b_post: status = "ERROR"
        elif a == b_pre and a != b_post: status = "BROKE"
        
        print(f"{i:<4} | {a:<5} | {b_pre:<10} | {b_post:<10} | {status}")

print_final_comparison(alice_server.sifted_key, bob_pre_ec, bob_client.working_key)

Idx  | Alice | Bob Pre-EC | Bob Final  | Status
-------------------------------------------------------
0    | 0     | 0          | 0          | OK
1    | 1     | 1          | 1          | OK
2    | 0     | 0          | 0          | OK
3    | 1     | 1          | 1          | OK
4    | 0     | 0          | 0          | OK
5    | 1     | 1          | 1          | OK
6    | 0     | 0          | 0          | OK
7    | 0     | 1          | 0          | FIXED
8    | 1     | 1          | 1          | OK
9    | 1     | 1          | 1          | OK
10   | 0     | 0          | 0          | OK
11   | 0     | 0          | 0          | OK
12   | 1     | 1          | 1          | OK
13   | 1     | 1          | 1          | OK
14   | 0     | 0          | 0          | OK
15   | 0     | 0          | 0          | OK
16   | 0     | 0          | 0          | OK
17   | 0     | 1          | 0          | FIXED
18   | 1     | 1          | 1          | OK
19   | 1     | 1          | 1          | OK
20   | 1  